# 텐서보드와 모델 재사용

### 학습 모델 저장후 재사용

In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

# iris 데이터로 실습
# print(data.DESCR) 설명을 보고 싶다면
x_data = datasets.load_iris().data
y_data = datasets.load_iris().target

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_data = y_data.reshape(len(y_data), 1)
y_data = onehot_encoder.fit_transform(y_data)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=2019)

C:\Users\PC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


### 신경망 모델 구성

In [16]:
# 학습 횟수를 카운트 하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([x_train.shape[1], 10], -1., 1.))
b1 = tf.Variable(tf.zeros([10]))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1),b1))

W2 = tf.Variable(tf.random_uniform([10, 5], -1., 1.))
b2 = tf.Variable(tf.zeros([5]))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2),b2))

W3 = tf.Variable(tf.random_uniform([5, 3], -1., 1.))
b3 = tf.Variable(tf.zeros([3]))

model = tf.add(tf.matmul(L2, W3),b3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
train_op = optimizer.minimize(cost, global_step=global_step)

In [18]:
with tf.Session() as sess:
    # tf.global_variables는 앞서 정의한 변수들을 가져오는 함수
    saver = tf.train.Saver(tf.global_variables())
    # ./model 디렉토리에 기존에 학습해둔 모델이 있는지 확인
    ckpt = tf.train.get_checkpoint_state('./model')
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
        
    for step in range(3000):
        sess.run(train_op, feed_dict={X: x_train, Y: y_train})
        if (step + 1) % 1000 == 0:
            print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_train, Y: y_train}))
        
    # 최적화가 끝난 뒤 학습 변수들 저장
    saver.save(sess, './model/dnn.ckpt', global_step=global_step)
    
    prediction = tf.argmax(model, axis = 1)
    target = tf.argmax(Y, axis=1)
    print('예측값: ', sess.run(prediction,feed_dict={X: x_test}))
    print('실제값: ', sess.run(target,feed_dict={Y: y_test}))

    # 정확도 출력
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도 %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

Step: 1000,  Cost: 0.059
Step: 2000,  Cost: 0.054
Step: 3000,  Cost: 0.051
예측값:  [0 0 2 1 2 0 2 0 1 2 2 2 2 1 0 1 2 1 0 2 0 2 0 1 0 0 1 2 0 0 2 0 0 2 2 0 0
 2 0 1 0 1 0 2 1]
실제값:  [0 0 2 1 2 0 2 0 1 2 2 2 2 1 0 1 2 1 0 2 0 2 0 1 0 0 1 2 0 0 2 0 0 2 2 0 0
 2 0 1 0 1 0 2 1]
정확도 98.67


# 텐서보드 사용
- 학습과정을 추적하려면 번거로운 작업을 많이 해야함
- 이런 문제를 해결하기 위해, 학습중간 손실값, 정확도, 결과물로 나온 이미지나 사운드 파일들을 시각화해서 보여줌

### 사용방법
1. 신경망의 각 계층에 다음 코드를 덧 붙임
```python
with tf.name_scope('[name지정]'):
```
2. 이후 변수 뒤에 이름을 붙임<br>
    텐서보드에서 해당 이름의 변수가 어디서 사용되는지 쉽게 확인가능

In [ ]:
# 학습 횟수를 카운트 하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([x_train.shape[1], 10], -1., 1.),name='W1')
    b1 = tf.Variable(tf.zeros([10]))
    L1 = tf.nn.relu(tf.add(tf.matmul(X, W1),b1))
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 5], -1., 1.),name='W2')
    b2 = tf.Variable(tf.zeros([5]))
    L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2),b2))
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([5, 3], -1., 1.),name='W3')
    b3 = tf.Variable(tf.zeros([3]))
    model = tf.add(tf.matmul(L2, W3),b3)
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.05)
    train_op = optimizer.minimize(cost, global_step=global_step)
    # 손실값을 추적하기 위해 수집할 값을 지정하는 코드
    tf.summary.scalar('cost',cost)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 지정한 텐서 수집
    merged = tf.summary.merge_all()
    # 그래프와 텐서들의 값 저장
    writer = tf.summary.FileWriter('./logs', sess.graph)
    
    for step in range(10000):
        sess.run(train_op, feed_dict={X: x_train, Y: y_train})
        summary = sess.run(merged, feed_dict={X:x_train, Y: y_train})
        writer.add_summary(summary, global_step=sess.run(global_step))
        if (step + 1) % 1000 == 0:
            print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_train, Y: y_train}))
        
    prediction = tf.argmax(model, axis = 1)
    target = tf.argmax(Y, axis=1)
    print('예측값: ', sess.run(prediction,feed_dict={X: x_test}))
    print('실제값: ', sess.run(target,feed_dict={Y: y_test}))

    # 정확도 출력
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도 %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

### Anaconda 명령 프롬포트에서 아래와 같이 입력
작업중인 폴더로 먼저 이동후에 실행해야됨

- cd "Desktop\TIL\ML\3min DL"
- tensorboard --logdir=./logs
- http://localhost:6006

#### 각 가중치와 편향 등의 변화를 그래프로 살펴보고 싶을때
- 텐서보드의 [DISTRIBUTIONS], [HISTOGRAMS] 메뉴 확인
- tf.summary.histogram("Weights", W1)

구글에서는 텐서플로로 만들고 학습시킨 모델을 실제 서비스에 적용하기 쉽게 만들어주는 서버환경인 텐서플로 서빙(Tensorflow Serving)을 제공

학습된 모델을 사용하는 프로그램을 텐서플로로 직접 만들수 있겠지만

쉽게 모델을 변경하거나, 여러 모델을 한 서버에서 서비스하는 등의 다양한 편의 기능을 제공함